In [1]:
# Importing dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time

In [2]:
# Using the Chrome Web driver and creating a browser object
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Visiting the Mars news website to scrape
url = "https://mars.nasa.gov/news/"
browser.visit(url)

# Waiting to ensure page has fully loaded
time.sleep(1)

In [4]:
# Scraping the page using BeautifulSoup
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
# Searching through the HTML for the title and first paragraph
results = soup.find_all("ul", class_ = "item_list")

news_title = results[0].find("div", class_ = "content_title").get_text()
news_p = results[0].find("div", class_ = "article_teaser_body").get_text()

In [6]:
# Printing the results to ensure they are correct
print(f"The latest news article is:\n{news_title}\n{news_p}")

The latest news article is:
Mars Is Getting a New Robotic Meteorologist
Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles.


In [7]:
# Visiting the NASA space images website to scrape
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

# Waiting to ensure page has fully loaded
time.sleep(1)

In [8]:
# Scraping the page using BeautifulSoup
html = browser.html
soup = bs(html, 'html.parser')

In [9]:
# Searching through the HTML for the image URL
results = soup.find("article")

img_url = "https://www.jpl.nasa.gov"

featured_image_url = img_url + results["style"].split("'")[1]

In [10]:
# Printing the results to ensure they are correct
print(f"The featured image URL is:\n{featured_image_url}")

The featured image URL is:
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17932-1920x1200.jpg


In [11]:
# Storing the space facts URL so that Pandas can read the table
url = "https://space-facts.com/mars/"

mars_df = pd.read_html(url)[0]
mars_df.columns = [" ", "Mars"]

mars_df.to_html("templates/mars_table.html", index = False)

In [12]:
# Visiting the Astrogeology website to scrape images
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

# Waiting to ensure page has fully loaded
time.sleep(1)

In [13]:
# Scraping the page using BeautifulSoup
html = browser.html
soup = bs(html, 'html.parser')

In [14]:
# Finding the title of each page we want to visit
results = soup.find_all("div", class_ = "item")

page_titles = []

for i in range(0, 4):
    page_titles.append(results[i].find("h3").get_text())
    
page_titles

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [15]:
# Using Splinter to visit each page and collect the image URL
img_url = []

for i in range(0, 4):
    try:
        browser.click_link_by_partial_text(page_titles[i])
        
        html = browser.html
        soup = bs(html, 'html.parser')
        
        results = soup.find_all("div", class_ = "downloads")[0]
        img_url.append(results.find("a")["href"])
        
        browser.back()
    except:
        print("Error, page not found")
        
img_url

C:\Users\matt_\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [16]:
# Merging the two previous lists into a list of dictionaries
hemisphere_image_urls = []

for i in range(0, 4):
    hemisphere_image_urls.append({"title": page_titles[i], "img_url": img_url[i]})
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [17]:
# Closing the browser now that scraping is complete
browser.quit()